In [ ]:
import logging
import importlib
importlib.reload(logging) # see https://stackoverflow.com/a/21475297/1469195
log = logging.getLogger()
log.setLevel('INFO')
import sys

logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)

In [ ]:
%%capture
import os
import site
os.sys.path.insert(0, '/home/schirrmr/code/reversible/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/explaining/reversible//')


%load_ext autoreload
%autoreload 2
import numpy as np
import logging
log = logging.getLogger()
log.setLevel('INFO')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png'
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 14
import seaborn
seaborn.set_style('darkgrid')

from reversible2.sliced import sliced_from_samples
from numpy.random import RandomState

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import copy
import math

import itertools
import torch as th
from braindecode.torch_ext.util import np_to_var, var_to_np
from reversible2.splitter import SubsampleSplitter

from reversible2.view_as import ViewAs

from reversible2.affine import AdditiveBlock
from reversible2.plot import display_text, display_close
from reversible2.high_gamma import load_file, create_inputs
from reversible2.high_gamma import load_train_test
th.backends.cudnn.benchmark = True
from reversible2.models import deep_invertible


In [ ]:
sensor_names = ['Fz', 
                'FC3','FC1','FCz','FC2','FC4',
                'C5','C3','C1','Cz','C2','C4','C6',
                'CP3','CP1','CPz','CP2','CP4',
                'P1','Pz','P2',
                'POz']

In [ ]:
# create model
# create dist

train_inputs, test_inputs = load_train_test(
    subject_id=4,
    car=True,
    n_sensors=22,
    final_hz=256,
    start_ms=500,
    stop_ms=1500,
    half_before=True,
    only_load_given_sensors=False,
)

In [ ]:
# create model
# create dist

test_dist_inputs, test_dist_inputs_2 = load_train_test(
    subject_id=5,
    car=True,
    n_sensors=22,
    final_hz=256,
    start_ms=500,
    stop_ms=1500,
    half_before=True,
    only_load_given_sensors=False,
)

In [ ]:
train_less = [t[:180,7:9].clone().contiguous() for t in train_inputs]
test_less = [t[:180,7:9].clone().contiguous() for t in test_inputs]
test_dist_less = [t[:180,7:9].clone().contiguous() for t in test_dist_inputs]
for t in train_less + test_less + test_dist_less:
    t.data[:,1] = 0

In [ ]:
from reversible2.models import larger_model

from reversible2.distribution import TwoClassIndependentDist

import ot

from reversible2.ot_exact import get_matched_samples


from reversible2.model_and_dist import ModelAndDist, set_dist_to_empirical
from reversible2.util import flatten_2d

In [ ]:
n_chans = train_less[0].shape[1]
n_time = train_less[0].shape[2]

n_chan_pad = 0
filter_length_time = 11
    
model = larger_model(n_chans, n_time, final_fft=True, kernel_length=11, constant_memory=False)
model.cuda()
from reversible2.models import add_dropout_before_convs
#add_dropout_before_convs(model,p_conv=0.3, p_full=0.3)
dist = TwoClassIndependentDist(np.prod(train_less[0].size()[1:]))
dist.cuda()
model_and_dist = ModelAndDist(model, dist)
model_and_dist.set_dist_to_empirical(train_less)

optim = th.optim.Adam([{'params': dist.parameters(), 'lr':1e-2},
                      {'params': list(model_and_dist.model.parameters()),
                      'lr': 1e-4}])

In [ ]:
from reversible2.model_and_dist import create_empirical_dist

In [ ]:
import pandas as pd
df = pd.DataFrame()


In [ ]:
samples = model_and_dist.dist.get_samples(i_class, 30).detach()
in_samples = model_and_dist.model.invert(samples)
perturbations = th.rand()

out_diffs = th.norm(samples.unsqueeze(0) - samples.unsqueeze(1), p=2, dim=2)

in_diffs = th.norm(flatten_2d(in_samples).unsqueeze(0) - flatten_2d(in_samples).unsqueeze(1), p=2, dim=2)

out_diffs = out_diffs.flatten()
in_diffs = in_diffs.flatten()

assert len(out_diffs.shape) == 1
assert len(in_diffs.shape) == 1
ratio = in_diffs / th.clamp(out_diffs, min=1e-9)
assert len(ratio.shape) == 1
lip_loss = th.mean(F.relu(ratio - lip_threshold) ** 2)


In [ ]:
from reversible2.timer import Timer
from reversible2.distribution import TwoClassIndependentDist
i_class = 1
n_epochs = 201
class_ins = train_less[i_class].cuda()
test_ins = test_less[i_class].cuda()
test_dist_ins = test_dist_less[i_class].cuda()
noise_factor = 1
lip_threshold = 1.3
lip_perturb_factor = 0.1
lip_loss_factor = 1000
for i_epoch in range(n_epochs):
    with Timer(verbose=False) as timer:
        if i_epoch > 0:
            model.train()
            optim.zero_grad()
            for i_class in range(2):
                class_ins = train_less[i_class].cuda()
                log_probs = model_and_dist.get_total_log_prob(
                    i_class, class_ins + (th.randn_like(class_ins)) * noise_factor)
                loss = -th.mean(log_probs)
                loss.backward()


            optim.step()
            model.eval()
            
    if i_epoch % 10 == 0:
        model.eval()
        with th.no_grad():
            samples = model_and_dist.dist.get_samples(i_class, 30).detach()
            in_samples = model_and_dist.model.invert(samples)

            out_diffs = th.norm(samples.unsqueeze(0) - samples.unsqueeze(1), p=2, dim=2)

            in_diffs = th.norm(flatten_2d(in_samples).unsqueeze(0) - flatten_2d(in_samples).unsqueeze(1), p=2, dim=2)

            out_diffs = out_diffs.flatten()
            in_diffs = in_diffs.flatten()

            assert len(out_diffs.shape) == 1
            assert len(in_diffs.shape) == 1
            ratio = in_diffs / th.clamp(out_diffs, min=1e-9)
            assert len(ratio.shape) == 1
            lip_loss = th.mean(F.relu(ratio - lip_threshold) ** 2)

            epoch_row = {'lip_loss': lip_loss.item()}
        
        with th.no_grad():
            for setname, inputs in (("Train", train_less), ("Test", test_less), ("Other", test_dist_less)):
                    OTs = []
                    nlls = []
                    inputs = [i.cuda() for i in inputs]
                    for i_class in range(2):
                        examples = model_and_dist.get_examples(i_class,len(inputs[i_class]) * 20)
                        matched_examples = get_matched_samples(flatten_2d(inputs[i_class]), flatten_2d(examples))
                        OT = th.mean(th.norm(flatten_2d(inputs[i_class]).unsqueeze(1)  - matched_examples, p=2, dim=2))#
                        nll = -th.mean(model_and_dist.get_total_log_prob(i_class, inputs[i_class]))
                        OTs.append(OT.item())
                        nlls.append(nll.item())
                    epoch_row[setname + '_OT'] = np.mean(OTs)
                    epoch_row[setname + '_NLL'] = np.mean(nlls)

            for setname, inputs in (("Train", train_less), ("Test", test_less)):
                    corrects = []
                    inputs =[i.cuda() for i in inputs]
                    for i_class in range(2):
                        outs = model_and_dist.log_softmax(inputs[i_class].cuda())
                        pred_label = np.argmax(var_to_np(outs), axis=1)
                        correct = pred_label == i_class
                        corrects.append(correct)
                    acc = np.mean(np.concatenate(corrects))
                    epoch_row['model_' + setname + '_acc'] = acc
        for name, inputs in (("Train", train_less),
                                 ("Combined", [th.cat((train_less[i_class].cuda(),
                                                    test_less[i_class].cuda()), dim=0)
                            for i_class in range(2)]),
                                 ("Test", test_less)):
                emp_dist = create_empirical_dist(model_and_dist.model, inputs)

                emp_model_dist = ModelAndDist(model_and_dist.model, emp_dist)
                with th.no_grad():
                    for setname, inner_inputs in (("Train", train_less), ("Test", test_less)):
                        inner_inputs =[i.cuda() for i in inner_inputs]
                        corrects = []
                        for i_class in range(2):
                            outs = emp_model_dist.log_softmax(inner_inputs[i_class].cuda())
                            pred_label = np.argmax(var_to_np(outs), axis=1)
                            correct = pred_label == i_class
                            corrects.append(correct)
                        acc = np.mean(np.concatenate(corrects))
                        epoch_row[name + '_' + setname + '_acc'] = acc
                        
        df = df.append(epoch_row, ignore_index=True)
                        

    if i_epoch % (n_epochs // 20) == 0:
        with th.no_grad():
            print("Epoch {:d} of {:d}".format(i_epoch, n_epochs))
            print("Runtime {:.1E} ms".format(timer.elapsed))
            text_strs = []
            for setname, inputs in (("Train", class_ins), ("Test", test_ins), ("Other", test_dist_ins)):
                inputs =  inputs.cuda()
                examples = model_and_dist.get_examples(1,len(inputs) * 20)
                matched_examples = get_matched_samples(flatten_2d(inputs), flatten_2d(examples))
                OT = th.mean(th.norm(flatten_2d(inputs).unsqueeze(1)  - matched_examples, p=2, dim=2))#
                nll = -th.mean(model_and_dist.get_total_log_prob(i_class, inputs))
                text_strs.append("{:7s} NLL {:.1E}".format(setname, nll.item()))
                text_strs.append("{:7s} OT {:.1E}".format(setname, OT.item()))
            display_text("\n".join(text_strs))
            
            print("Actual Model")
            for setname, inputs in (("Train", train_less), ("Test", test_less)):
                inputs =[i.cuda() for i in inputs]
                corrects = []
                for i_class in range(2):
                    outs = model_and_dist.log_softmax(inputs[i_class].cuda())
                    pred_label = np.argmax(var_to_np(outs), axis=1)
                    correct = pred_label == i_class
                    corrects.append(correct)
                acc = np.mean(np.concatenate(corrects))
                print("{:6s} Accuracy {:.1f} ({:.1f}/{:.1f})".format(setname, acc * 100,
                      np.mean(corrects[0]) * 100,
                      np.mean(corrects[1]) * 100))



            for name, inputs in (("Train", train_less),
                                 ("Combined", [th.cat((train_less[i_class].cuda(),
                                                    test_less[i_class].cuda()), dim=0)
                            for i_class in range(2)]),
                                 ("Test", test_less)):
                
                inputs =[i.cuda() for i in inputs]
                emp_dist = create_empirical_dist(model_and_dist.model, inputs)

                emp_model_dist = ModelAndDist(model_and_dist.model, emp_dist)
                print(name)
                with th.no_grad():
                    for setname, inner_inputs in (("Train", train_less), ("Test", test_less)):
                        corrects = []
                        inner_inputs =[i.cuda() for i in inner_inputs]
                        for i_class in range(2):
                            outs = emp_model_dist.log_softmax(inner_inputs[i_class].cuda())
                            pred_label = np.argmax(var_to_np(outs), axis=1)
                            correct = pred_label == i_class
                            corrects.append(correct)
                        acc = np.mean(np.concatenate(corrects))
                        print("{:6s} Accuracy {:.1f} ({:.1f}/{:.1f})".format(setname, acc * 100,
                                                                          np.mean(corrects[0]) * 100,
                                                                          np.mean(corrects[1]) * 100))
            examples = model_and_dist.get_examples(1,len(class_ins) * 20)
            matched_examples = get_matched_samples(flatten_2d(class_ins), flatten_2d(examples))
            fig, axes = plt.subplots(5,2, figsize=(16,12), sharex=True, sharey=True)
            for ax, signal, matched in zip(axes.flatten(), class_ins, matched_examples):
                ax.plot(var_to_np(signal).squeeze().T)
                for ex in var_to_np(matched.view(len(matched), class_ins.shape[1], class_ins.shape[2])):
                    ax.plot(ex[0], color=seaborn.color_palette()[0], lw=0.5, alpha=0.7)
                    ax.plot(ex[1], color=seaborn.color_palette()[1], lw=0.5, alpha=0.7)
            display_close(fig)
            fig = plt.figure()
            plt.plot(var_to_np(th.exp(model_and_dist.dist.class_log_stds)[1]))
            display_close(fig)
            examples = model_and_dist.get_examples(1,len(class_ins) * 20)
            fake_bps = np.abs(np.fft.rfft(var_to_np(examples[:,0]).squeeze()))
            real_bps = np.abs(np.fft.rfft(var_to_np(class_ins[:,0]).squeeze()))
            fig = plt.figure(figsize=(8,3))
            plt.plot(np.fft.rfftfreq(256, 1/256.0), np.mean(real_bps, axis=0))
            plt.plot(np.fft.rfftfreq(256, 1/256.0), np.mean(fake_bps, axis=0))
            display_close(fig)

### split off 40 validation trials

In [ ]:

valid_less = [t[-40:] for t in train_less]
train_less = [t[:-40] for t in train_less]

In [ ]:
n_chans = train_less[0].shape[1]
n_time = train_less[0].shape[2]



n_chan_pad = 0
filter_length_time = 11
    
model = larger_model(n_chans, n_time, final_fft=True, kernel_length=11, constant_memory=False)
model.cuda()
from reversible2.models import add_dropout_before_convs
#add_dropout_before_convs(model,p_conv=0.3, p_full=0.3)
dist = TwoClassIndependentDist(np.prod(train_less[0].size()[1:]))
dist.cuda()
model_and_dist = ModelAndDist(model, dist)
model_and_dist.set_dist_to_empirical(train_less)

optim = th.optim.Adam([{'params': dist.parameters(), 'lr':1e-2},
                      {'params': list(model_and_dist.model.parameters()),
                      'lr': 1e-4}])
rand_log_noise_factor = th.zeros(1, requires_grad=True, device='cuda')
optim_noise = th.optim.Adam([
                       {'params': [rand_log_noise_factor], 'lr':1e-2},])

In [ ]:
from reversible2.timer import Timer
from reversible2.distribution import TwoClassIndependentDist
from reversible2.mixture import GaussianMixture
n_epochs = 201
class_ins = train_less[i_class].cuda()
test_ins = test_less[i_class].cuda()
test_dist_ins = test_dist_less[i_class].cuda()
noise_factor = 1
lip_threshold = 1.3
lip_perturb_factor = 0.1
lip_loss_factor = 1000
for i_epoch in range(n_epochs):
    with Timer(verbose=False) as timer:
        if i_epoch > 0:
            model.train()
            optim.zero_grad()
            for i_class in range(2):
                class_ins = train_less[i_class].cuda()
                log_probs = model_and_dist.get_total_log_prob(
                    i_class, class_ins + (th.randn_like(class_ins)) * th.exp(rand_log_noise_factor))
                loss = -th.mean(log_probs)
                loss.backward()


            optim.step()
            optim_noise.zero_grad()
            for i_class in range(2):
                train_ins = train_less[i_class].cuda()
                valid_ins = valid_less[i_class].cuda()
                
                with th.no_grad():
                    tr_outs = model_and_dist.model(train_ins).detach()
                log_stds = rand_log_noise_factor.repeat(tr_outs.shape)
                mixture = GaussianMixture(tr_outs, log_stds)
                with th.no_grad():
                    val_outs = model_and_dist.model(valid_ins).detach()
                nll = -th.mean(mixture.log_probs(val_outs))
                nll.backward()

            optim_noise.step()
            
            
        model.eval()

            
    if i_epoch % 10 == 0:
        model.eval()
        with th.no_grad():
            samples = model_and_dist.dist.get_samples(i_class, 30).detach()
            in_samples = model_and_dist.model.invert(samples)

            out_diffs = th.norm(samples.unsqueeze(0) - samples.unsqueeze(1), p=2, dim=2)

            in_diffs = th.norm(flatten_2d(in_samples).unsqueeze(0) - flatten_2d(in_samples).unsqueeze(1), p=2, dim=2)

            out_diffs = out_diffs.flatten()
            in_diffs = in_diffs.flatten()

            assert len(out_diffs.shape) == 1
            assert len(in_diffs.shape) == 1
            ratio = in_diffs / th.clamp(out_diffs, min=1e-9)
            assert len(ratio.shape) == 1
            lip_loss = th.mean(F.relu(ratio - lip_threshold) ** 2)

            epoch_row = {'lip_loss': lip_loss.item()}
        
        with th.no_grad():
            for setname, inputs in (("Train", train_less), ("Test", test_less), ("Other", test_dist_less)):
                    OTs = []
                    nlls = []
                    inputs = [i.cuda() for i in inputs]
                    for i_class in range(2):
                        examples = model_and_dist.get_examples(i_class,len(inputs[i_class]) * 20)
                        matched_examples = get_matched_samples(flatten_2d(inputs[i_class]), flatten_2d(examples))
                        OT = th.mean(th.norm(flatten_2d(inputs[i_class]).unsqueeze(1)  - matched_examples, p=2, dim=2))#
                        nll = -th.mean(model_and_dist.get_total_log_prob(i_class, inputs[i_class]))
                        OTs.append(OT.item())
                        nlls.append(nll.item())
                    epoch_row[setname + '_OT'] = np.mean(OTs)
                    epoch_row[setname + '_NLL'] = np.mean(nlls)

            for setname, inputs in (("Train", train_less), ("Test", test_less)):
                    corrects = []
                    inputs =[i.cuda() for i in inputs]
                    for i_class in range(2):
                        outs = model_and_dist.log_softmax(inputs[i_class].cuda())
                        pred_label = np.argmax(var_to_np(outs), axis=1)
                        correct = pred_label == i_class
                        corrects.append(correct)
                    acc = np.mean(np.concatenate(corrects))
                    epoch_row['model_' + setname + '_acc'] = acc
        for name, inputs in (("Train", train_less),
                                 ("Combined", [th.cat((train_less[i_class].cuda(),
                                                    test_less[i_class].cuda()), dim=0)
                            for i_class in range(2)]),
                                 ("Test", test_less)):
                emp_dist = create_empirical_dist(model_and_dist.model, inputs)

                emp_model_dist = ModelAndDist(model_and_dist.model, emp_dist)
                with th.no_grad():
                    for setname, inner_inputs in (("Train", train_less), ("Test", test_less)):
                        inner_inputs =[i.cuda() for i in inner_inputs]
                        corrects = []
                        for i_class in range(2):
                            outs = emp_model_dist.log_softmax(inner_inputs[i_class].cuda())
                            pred_label = np.argmax(var_to_np(outs), axis=1)
                            correct = pred_label == i_class
                            corrects.append(correct)
                        acc = np.mean(np.concatenate(corrects))
                        epoch_row[name + '_' + setname + '_acc'] = acc
                        
        df = df.append(epoch_row, ignore_index=True)
                        

    if i_epoch % (n_epochs // 20) == 0:
        with th.no_grad():
            print("Epoch {:d} of {:d}".format(i_epoch, n_epochs))
            print("Runtime {:.1E} ms".format(timer.elapsed))
            text_strs = []
            for setname, inputs in (("Train", class_ins), ("Test", test_ins), ("Other", test_dist_ins)):
                inputs =  inputs.cuda()
                examples = model_and_dist.get_examples(1,len(inputs) * 20)
                matched_examples = get_matched_samples(flatten_2d(inputs), flatten_2d(examples))
                OT = th.mean(th.norm(flatten_2d(inputs).unsqueeze(1)  - matched_examples, p=2, dim=2))#
                nll = -th.mean(model_and_dist.get_total_log_prob(i_class, inputs))
                text_strs.append("{:7s} NLL {:.1E}".format(setname, nll.item()))
                text_strs.append("{:7s} OT {:.1E}".format(setname, OT.item()))
            display_text("\n".join(text_strs))
            
            print("Actual Model")
            for setname, inputs in (("Train", train_less), ("Test", test_less)):
                inputs =[i.cuda() for i in inputs]
                corrects = []
                for i_class in range(2):
                    outs = model_and_dist.log_softmax(inputs[i_class].cuda())
                    pred_label = np.argmax(var_to_np(outs), axis=1)
                    correct = pred_label == i_class
                    corrects.append(correct)
                acc = np.mean(np.concatenate(corrects))
                print("{:6s} Accuracy {:.1f} ({:.1f}/{:.1f})".format(setname, acc * 100,
                      np.mean(corrects[0]) * 100,
                      np.mean(corrects[1]) * 100))



            for name, inputs in (("Train", train_less),
                                 ("Combined", [th.cat((train_less[i_class].cuda(),
                                                    test_less[i_class].cuda()), dim=0)
                            for i_class in range(2)]),
                                 ("Test", test_less)):
                
                inputs =[i.cuda() for i in inputs]
                emp_dist = create_empirical_dist(model_and_dist.model, inputs)

                emp_model_dist = ModelAndDist(model_and_dist.model, emp_dist)
                print(name)
                with th.no_grad():
                    for setname, inner_inputs in (("Train", train_less), ("Test", test_less)):
                        corrects = []
                        inner_inputs =[i.cuda() for i in inner_inputs]
                        for i_class in range(2):
                            outs = emp_model_dist.log_softmax(inner_inputs[i_class].cuda())
                            pred_label = np.argmax(var_to_np(outs), axis=1)
                            correct = pred_label == i_class
                            corrects.append(correct)
                        acc = np.mean(np.concatenate(corrects))
                        print("{:6s} Accuracy {:.1f} ({:.1f}/{:.1f})".format(setname, acc * 100,
                                                                          np.mean(corrects[0]) * 100,
                                                                          np.mean(corrects[1]) * 100))
            examples = model_and_dist.get_examples(1,len(class_ins) * 20)
            matched_examples = get_matched_samples(flatten_2d(class_ins), flatten_2d(examples))
            fig, axes = plt.subplots(5,2, figsize=(16,12), sharex=True, sharey=True)
            for ax, signal, matched in zip(axes.flatten(), class_ins, matched_examples):
                ax.plot(var_to_np(signal).squeeze().T)
                for ex in var_to_np(matched.view(len(matched), class_ins.shape[1], class_ins.shape[2])):
                    ax.plot(ex[0], color=seaborn.color_palette()[0], lw=0.5, alpha=0.7)
                    ax.plot(ex[1], color=seaborn.color_palette()[1], lw=0.5, alpha=0.7)
            display_close(fig)
            fig = plt.figure()
            plt.plot(var_to_np(th.exp(model_and_dist.dist.class_log_stds)[1]))
            display_close(fig)
            examples = model_and_dist.get_examples(1,len(class_ins) * 20)
            fake_bps = np.abs(np.fft.rfft(var_to_np(examples[:,0]).squeeze()))
            real_bps = np.abs(np.fft.rfft(var_to_np(class_ins[:,0]).squeeze()))
            fig = plt.figure(figsize=(8,3))
            plt.plot(np.fft.rfftfreq(256, 1/256.0), np.mean(real_bps, axis=0))
            plt.plot(np.fft.rfftfreq(256, 1/256.0), np.mean(fake_bps, axis=0))
            display_close(fig)

In [ ]:
other_model_dist = th.load('/data/schirrmr/schirrmr/reversible/experiments/dropout-weight-decay/243/model_and_dist.pkl')

In [ ]:
out = model_and_dist.model(train_less[1][:1].cuda())

In [ ]:
del outs

In [ ]:
class_ins = train_inputs[1].cuda()

fig = plt.figure()
plt.plot(var_to_np(th.exp(other_model_dist.dist.class_log_stds)[1]))
display_close(fig)
examples = other_model_dist.get_examples(1,len(class_ins) * 1)
fake_bps = np.abs(np.fft.rfft(var_to_np(examples[:,0]).squeeze()))
fig = plt.figure(figsize=(8,3))
plt.plot(np.fft.rfftfreq(256, 1/256.0), np.mean(real_bps, axis=0))
plt.plot(np.fft.rfftfreq(256, 1/256.0), np.mean(fake_bps, axis=0))
display_close(fig)

In [ ]:
for i_class in range(2):
    class_ins = train_less[i_class].cuda()
    log_probs = model_and_dist.get_total_log_prob(
        i_class, class_ins + (th.randn_like(class_ins)) * th.exp(rand_log_noise_factor))
    loss = -th.mean(log_probs)
    print(loss)
    

In [ ]:
mixtures = []
for i_class in range(2):
    train_ins = train_less[i_class].cuda()
    with th.no_grad():
        tr_outs = model_and_dist.model(train_ins).detach()
    log_stds = rand_log_noise_factor.repeat(tr_outs.shape)
    mixture = GaussianMixture(tr_outs, log_stds)
    mixtures.append(mixture)
            

for i_class in range(2):
    test_ins = test_less[i_class].cuda()
    with th.no_grad():
        te_outs = model_and_dist.model(test_ins).detach()
    log_probs = [m.log_probs(te_outs) for m in mixtures]
    print(np.mean(np.argmax(var_to_np(th.stack(log_probs, dim=1)), axis=1) == i_class))

In [ ]:
th.exp(rand_log_noise_factor)

In [ ]:
for i_class in range(2):
    class_ins = train_less[i_class].cuda()
    log_probs = model_and_dist.get_total_log_prob(
        1-i_class, class_ins + (th.randn_like(class_ins)) * th.exp(rand_log_noise_factor))
    loss = -th.mean(log_probs)
    print(loss)
    

In [ ]:
plt.plot(var_to_np(dist.get_mean_std(0)[0]))
plt.plot(var_to_np(dist.get_mean_std(1)[0]))


In [ ]:
tr_out = model_and_dist.model(train_less[0].cuda())
plt.plot(np.mean(var_to_np(tr_out), axis=0))
tr_out = model_and_dist.model(train_less[1].cuda())
plt.plot(np.mean(var_to_np(tr_out), axis=0))

In [ ]:
with th.no_grad():
    for i_class in range(2):
        tr_out = model_and_dist.model(train_less[i_class].cuda())
        plt.plot(np.mean(var_to_np(tr_out), axis=0))
    for i_class in range(2):
        tr_out = model_and_dist.model(valid_less[i_class].cuda())
        plt.plot(np.mean(var_to_np(tr_out), axis=0))

In [ ]:
plt.plot(var_to_np(dist.get_mean_std(0)[1]))
plt.plot(var_to_np(dist.get_mean_std(1)[1]))

In [ ]:
dist.get_mean_std(1)

In [ ]:
inputs =[i.cuda() for i in train_less]
emp_dist = create_empirical_dist(model_and_dist.model, inputs)

emp_model_dist = ModelAndDist(model_and_dist.model, emp_dist)

In [ ]:
plt.plot(var_to_np(emp_model_dist.dist.get_mean_std(0)[0]))
plt.plot(var_to_np(emp_model_dist.dist.get_mean_std(1)[0]))

In [ ]:
emp_model_dist.dist.get_mean_std(0)[0]

In [ ]:
p 0,0.1,0.3,0.5
w decay 0 1e-4, 1e-3, 1e-2,
noise factor 1e-2, 5e-2, 1e-1